In [ ]:
from email.policy import default
import sys, os

import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import cell2location
import scvi
from scvi import REGISTRY_KEYS
import matplotlib
from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42 
import seaborn as sns
from optparse import OptionParser

In [ ]:
class Cell2locationAnno:
    def __init__(self, ref_file, adata_vis_file, outdir, gpu=1, bin_type = 'cell'):
        self.ref_file = ref_file
        self.adata_vis_file = adata_vis_file
        self.results_folder = outdir
        self.gpu = f"cuda:{gpu}"
        self.spot_size = 70
        self.bin_type = bin_type
        self.ref_run_name = f'{self.results_folder}/reference_signatures'
        self.run_name = f'{self.results_folder}/cell2location_map_{self.bin_type}'
        self.figures = f'{self.results_folder}/figures_{self.bin_type}'
        os.makedirs(self.ref_run_name, exist_ok=True)
        os.makedirs(self.run_name, exist_ok=True)
        os.makedirs(self.figures, exist_ok=True)
        sc.settings.figdir = self.figures

    def run_anno(self, N_cells_per_location):
        if self.ref_file.endswith(".h5ad"):
            inf_aver = self.process_ref()
        elif self.ref_file.endswith(".csv"):
            inf_aver = pd.read_csv(self.ref_file, index_col=0)
        adata_vis  = self.process_vis(inf_aver, N_cells_per_location=N_cells_per_location)

        return adata_vis

    def process_ref(self, batch_key = 'tissue', labels_key = 'leiden_anno'):
        sc.settings.figdir = self.figures
        inf_aver_file = f"{self.ref_run_name}/inf_aver.csv"
        if (os.path.exists(inf_aver_file)):
            inf_aver = pd.read_csv(inf_aver_file, index_col=0)
            return inf_aver
        elif (os.path.exists(f"{self.ref_run_name}/model.pt")):
            adata_ref_file = f"{self.ref_run_name}/sc.h5ad"
            adata_ref = sc.read_h5ad(adata_ref_file)
            mod = cell2location.models.RegressionModel.load(self.ref_run_name, adata_ref, use_gpu=self.gpu)
        else:
            adata_ref = sc.read_h5ad(self.ref_file)
            adata_ref.obs_names_make_unique()
            sc.pp.filter_cells(adata_ref, min_genes = 0)
            sc.pp.filter_genes(adata_ref, min_counts = 10)
            adata_ref.var['mt'] = adata_ref.var_names.str.startswith(('mt-', 'MT-'))
            sc.pp.calculate_qc_metrics(adata_ref, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
            sc.pl.violin(adata_ref, ['n_genes', 'total_counts', 'pct_counts_mt'], jitter=0.4, multi_panel=True, save='ref_violin.png')
            sc.pl.scatter(adata_ref, x='total_counts', y='n_genes', save = 'ref_scatter.png')

            from cell2location.utils.filtering import filter_genes
            selected = filter_genes(adata_ref, cell_count_cutoff=5, cell_percentage_cutoff2=0.03, nonz_mean_cutoff=1.12)
            plt.savefig(f"{self.figures}/ref_filter_genes.png")
            plt.clf()
            #filter the object
            adata_ref = adata_ref[:, selected].copy()
            #prepare anndata for the regression model
            cell2location.models.RegressionModel.setup_anndata(adata=adata_ref,
                                                   # 10X reaction / sample /batch
                                                   batch_key = batch_key,
                                                   #batch_key = 'Sample',
                                                   # cell type, covariate used fir contructing signatures
                                                   labels_key = labels_key,
                                                   #labels_key = 'Subset',
                                                   # multiplicative technical effects (platform, 3' vs 5', donor effect)
                                                   #categorical_covariate_keys = ['Experiment']
                                                   #categorical_covariate_keys = ['Method']
                                                  )
            # create the regression model
            from cell2location.models import RegressionModel
            mod = RegressionModel(adata_ref)
            #vies anndata_setup as a sanity check
            mod.view_anndata_setup()
            #os.environ['CUDA_VISIBLE_DEVICES'] = '1'
            #os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
            mod.train(max_epochs=1500, use_gpu=self.gpu)

            # In this section, we export the estimated cell abundance (summary of the posterior distribution).
            adata_ref = mod.export_posterior(
                adata_ref, sample_kwargs={'num_samples':1000, 'batch_size': 2500, 'use_gpu': True}
            )

            #Save model
            mod.save(f"{self.ref_run_name}", overwrite=True)

            #Save anndata object with results
            adata_file = f"{self.ref_run_name}/sc.h5ad"
            adata_ref.write(adata_file)
            mod.plot_history(20)
            plt.savefig(f"{self.figures}/ref_train_history.png")
            plt.clf()
            inf_aver = mod.samples[f"post_sample_means"]["per_cluster_mu_fg"].T
            if "detection_y_c" in list(mod.samples[f"post_sample_means"].keys()):
                inf_aver = inf_aver * mod.samples[f"post_sample_means"]["detection_y_c"].mean()
            aver = mod._compute_cluster_averages(key=REGISTRY_KEYS.LABELS_KEY)
            aver = aver[mod.factor_names_]
            plt.hist2d(np.log10(aver.values.flatten() + 1),
                        np.log10(inf_aver.flatten() + 1),
                        bins = 50,
                        norm = matplotlib.colors.LogNorm(),
            )
            plt.xlabel("Mean expression for every gene in every cluster")
            plt.ylabel("Estimated expression for every gene in every cluster")
            plt.savefig(f"{self.figures}/ref_train_QC.png")
            plt.clf()
            #export estimated expression in each cluster
        if 'means_per_cluster_mu_fg' in adata_ref.varm.keys():
            inf_aver = adata_ref.varm['means_per_cluster_mu_fg'][[f'means_per_cluster_mu_fg_{i}'
                                                          for i in adata_ref.uns['mod']['factor_names']]].copy()
        else:
            inf_aver = adata_ref.var[[f'means_per_cluster_mu_fg_{i}'
                                        for i in adata_ref.uns['mod']['factor_names']]].copy()
        inf_aver.columns = adata_ref.uns['mod']['factor_names']
        inf_aver.to_csv(inf_aver_file)
        return inf_aver

    def process_vis(self, inf_aver, N_cells_per_location=1, max_epochs=1200):
        sc.settings.figdir = self.figures
        # find shared genes and subset both anndata and reference signatures
        adata_vis = anndata.read(self.adata_vis_file)
        if adata_vis.raw != None:
            adata_vis = adata_vis.raw.to_adata()
        adata_vis.raw = adata_vis
        #find mitochondria-encoded (MT) genes
        adata_vis.var['MT_gene'] = adata_vis.var_names.str.startswith(('mt-', 'MT-'))

        # remove MT genes for spatial mapping (keeping their counts in the object)
        adata_vis.obsm['MT'] = adata_vis[:, adata_vis.var['MT_gene'].values].X.toarray()
        adata_vis = adata_vis[:, ~adata_vis.var['MT_gene'].values]
        intersect = np.intersect1d(adata_vis.var_names, inf_aver.index)
        adata_vis = adata_vis[:, intersect].copy()
        inf_aver = inf_aver.loc[intersect, :].copy()

        #prepare anndata for cell2location model
        cell2location.models.Cell2location.setup_anndata(adata=adata_vis)

        # create and train the model
        mod = cell2location.models.Cell2location(
            adata_vis, cell_state_df=inf_aver,
            # the expected average cell abundance: tissue-dependent
            # hyper-prior which can be estimated from paired histology:
            N_cells_per_location=N_cells_per_location,
            # hyperparameter controlling normalisation of
            # within-experiment variation in RNA detection:
            detection_alpha=20
        )
        mod.view_anndata_setup()
        #os.environ['CUDA_VISIBLE_DEVICES'] = '4'
        #mod.to_device('cuda:0')
        batch_size = None if mod.adata.n_obs < 75000 else 75000
        mod.train(max_epochs=max_epochs,
                # train using full data (batch_size=None)
                batch_size=10000,
                # use all data points in training because
                # we need to estimate cell abundance at all locations
                train_size=1,
                use_gpu=self.gpu)

        # In this section, we export the estimated cell abundance (summary of the posterior distribution).
        adata_vis = mod.export_posterior(
            adata_vis, sample_kwargs={'num_samples': 1000, 'batch_size': 7500, 'use_gpu': self.gpu}
        )

        # Save model
        mod.save(f"{self.run_name}", overwrite=True)
        mod.plot_history(20)
        plt.legend(labels=['full data training'])
        plt.savefig(f"{self.figures}/vis_train_history.png")
        plt.clf()

        use_n_obs = 1000
        ind_x = np.random.choice(mod.adata_manager.adata.n_obs, np.min((use_n_obs, mod.adata.n_obs)), replace=False)
        mod.expected_nb_param = mod.module.model.compute_expected(
            mod.samples[f"post_sample_means"], mod.adata_manager, ind_x=ind_x
        )
        x_data = mod.adata_manager.get_from_registry(REGISTRY_KEYS.X_KEY)[ind_x, :]
        x_data = np.asarray(x_data.toarray())
        mod.plot_posterior_mu_vs_data(mod.expected_nb_param["mu"], x_data)
        plt.savefig(f"{self.figures}/vis_QC.png")
        plt.clf()

        # Save anndata object with results
        adata_file = f"{self.run_name}/sp.h5ad"
        # adata_vis.write(adata_file)
        # add 5% quantile, representing confident cell abundance, 'at least this amount is present',
        # to adata.obs with nice names for plotting
        adata_vis.obs[adata_vis.uns['mod']['factor_names']] = adata_vis.obsm['q05_cell_abundance_w_sf']
        cellList=list(set(inf_aver.columns) & set(adata_vis.obs.columns))
        with mpl.rc_context({'axes.facecolor':  'black', 'figure.figsize': [5, 5]}):
            sc.pl.spatial(adata_vis, img_key="hires", color=cellList, spot_size=self.spot_size, vmin=0, vmax='p99.2', 
                        cmap='magma', save="cell_abundance.png")

        adata_vis.obs['cell2loc_anno'] = adata_vis.obs[cellList].idxmax(axis=1)
        
        # Save anndata object with results
        anno = 'cell2loc_anno'
        self._plotCells(anno, adata_vis, spot_size=self.spot_size)
        adata_vis.write(adata_file)
        return adata_vis

In [ ]:
class CellCluster():
    def __init__(self, out_file):
        self.out_file = out_file

    def scanpy_cluster(self, adata):

        sc.pp.filter_cells(adata, min_genes=0)
        sc.pp.filter_genes(adata, min_cells=0)
        adata.var['mt'] = adata.var_names.str.startswith(('mt-', 'MT-'))
        sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

        adata.raw = adata
        try:
            sc.pp.normalize_total(adata, target_sum=1e4)
            sc.pp.log1p(adata)
        except:
            print ("this data is not enough to do cell cluster.")
            exit(0)
        
        #sc.pp.highly_variable_genes(adata, flavor="seurat", n_top_genes=2000)
        #adata = adata[:, adata.var.highly_variable]
        
        sc.pp.scale(adata, max_value=10)
        sc.tl.pca(adata, svd_solver="arpack")
        sc.pp.neighbors(adata, n_neighbors=100, n_pcs=10)
        sc.tl.tsne(adata)
        sc.tl.umap(adata)
        sc.tl.leiden(adata, resolution=0.5)
        sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon', n_genes=200, key_added='rank_genes_groups_leiden', pts=True)
        try:
            adata.write(self.out_file)
        except:
            adata.uns.pop('rank_genes_groups_leiden')
            try:
                adata.write(self.out_file)
            except:
                print ("cluster file is too big to write....")
                exit(0)
        return adata

In [ ]:
def getOffset(tissueGem):
    
    offsetX = 0
    offsetY = 0
    reader = gzip.open(tissueGem, 'rt') if tissueGem.endswith(".gz") else open(tissueGem, 'r')
    for line in reader:
        if not line.startswith("#"):
            break
        elif "OffsetX" in line:
            offsetX = int(line.strip().split("=")[1])
        elif "OffsetY" in line:
            offsetY = int(line.strip().split("=")[1])
    reader.close()
    return offsetX, offsetY

def gem2h5ad(geneExpFile, binSize=50, tissueMask=None):
    print (f"bin size has been set to {binSize}")
    columntypes = {"geneID": 'category',
                   "geneName":"category", 
                   "x": int, 
                   'y': int, 
                   "UMICount": int, 
                   "MIDCount": int, 
                   "MIDCounts": int, 
                   "values": int
                  }
    df = pd.read_csv(geneExpFile, sep="\t", quoting=csv.QUOTE_NONE, comment="#", dtype=columntypes)
    
    if "MIDCounts" in df.columns:
        df.rename(columns={"MIDCounts": "UMICount"}, inplace=True)
    elif 'values' in df.columns:
        df.rename(columns={"values": "UMICount"}, inplace=True)
    elif 'MIDCount' in df.columns:
        df.rename(columns={'MIDCount': 'UMICount'}, inplace=True)
        
    if "geneName" in df.columns:
        df = df[["x","y","geneName","UMICount"]]
        df.rename(columns={'geneName': 'geneID'}, inplace=True)
        
    if tissueMask != None:
        offsetX, offsetY = getOffset(geneExpFile)
        df['x'] = df['x'] + offsetX
        df['y'] = df['y'] + offsetY
        mask = tifffile.imread(tissueMask)
        maxY, maxX = mask.shape
        if maxX > df['x'].max() or maxY > df['y'].max():
            print ("WARMING: mask is out of bounds")
            df = df.loc[(df['x'] < maxX)&(df['y'] < maxY)]
        df = df.loc[mask[df['y'], df['x']] > 0]
    
    halfBinSize = int(binSize/2)
    
    df['x'] = (df['x']//binSize)*binSize + halfBinSize
    df['y'] = (df['y']//binSize)*binSize + halfBinSize 
    df['cell'] = df['x'].astype(str) + "-" + df['y'].astype(str)

    cells = df['cell'].unique()
    genes = df['geneID'].unique()
    
    cells_dict = dict(zip(cells, range(0, len(cells))))
    genes_dict = dict(zip(genes, range(0, len(genes))))
    rows = df['cell'].map(cells_dict)
    cols = df['geneID'].map(genes_dict)
    expMtx = sparse.csr_matrix((df['UMICount'].values, (rows, cols)), 
                               shape=(cells.shape[0], genes.shape[0]), 
                               dtype=np.int32)

    obs = pd.DataFrame(index = cells)
    var = pd.DataFrame(index = genes)
    adata = anndata.AnnData(X = expMtx, obs = obs, var = var)
    positions = np.array(list(map(lambda x: [int(v) for v in x.strip().split("-")], adata.obs.index)))
    adata.obs['x'] = positions[:,0]
    adata.obs['y'] = positions[:,1]
    adata.obsm['spatial'] = adata.obs[['x', 'y']].values
    return adata

def cellbin2h5ad(geneExpFile):
    
    columntypes = {"geneID": object, 
                   "x": np.uint32, 
                   "y": np.uint32, 
                   "UMICount": np.uint16, 
                   "MIDCount": np.uint16, 
                   "label": np.uint32, 
                   "tag": "category"
                  }
    df = pd.read_csv(geneExpFile, sep="\t", dtype=columntypes)
    if "MIDCounts" in df.columns:
        df.rename(columns={"MIDCounts": "UMICount"}, inplace=True)
    elif 'values' in df.columns:
        df.rename(columns={"values": "UMICount"}, inplace=True)
    elif 'MIDCount' in df.columns:
        df.rename(columns={'MIDCount': 'UMICount'}, inplace=True)
    df = df[df['label']!=0]
    
    cells = df['label'].unique()
    genes = df['geneID'].unique()
    cells_dict = dict(zip(cells, range(0, len(cells))))
    genes_dict = dict(zip(genes, range(0, len(genes))))
    rows = df['label'].map(cells_dict)
    cols = df['geneID'].map(genes_dict)

    expMtx = sparse.csr_matrix((df['UMICount'].values, (rows, cols)), 
                               shape=(cells.shape[0], genes.shape[0]), 
                               dtype=np.int32)

    obs = pd.DataFrame(index = cells)
    var = pd.DataFrame(index = genes)
    adata = anndata.AnnData(X = expMtx, obs = obs, var = var)
    spatialgroup = df[['x', 'y']].groupby(df['label'])
    spatialdf = spatialgroup.agg(lambda x: (x.max()+x.min())/2)
    spatialdf = spatialdf.reset_index()
    spatialdf['label'] = spatialdf['label'].astype('category')
    spatialdf['label'].cat.reorder_categories(cells, inplace=True)
    spatialdf.sort_values('label', inplace=True)
    adata.obsm['spatial'] = spatialdf[['x', 'y']].values
    return adata

def _colorChange(value):
    
    digit = list(map(str, range(10))) + list("abcdef")
    a1 = digit.index(value[1]) * 16 + digit.index(value[2])
    a2 = digit.index(value[3]) * 16 + digit.index(value[4])
    a3 = digit.index(value[5]) * 16 + digit.index(value[6])
    return (a3, a2, a1)

def _getColorList(mycolor, colorTypes=30):
    
    if (mycolor == None):
        mycolor = ['#0343df', '#f97306', '#15b01a', '#e50000', '#7e1e9c', '#653700', '#ff81c0', '#929591', 
            '#6e750e', '#00ffff', '#ff796c', '#06c2ac', '#75bbfd', '#01ff07', '#cb416b', '#bf77f6', '#ceb301', 
            '#137e6d', '#516572', '#dbb40c', '#d0fefe', '#9ffeb0', '#fdaa48', '#ffcfdc', '#ffffc2', '#ac9362', 
            '#7a9703', '#96ae8d', '#b66a50', '#411900']
    rgb_mycolor = []
    for color in mycolor:
        rgb_mycolor.append(list(_colorChange(color)))
    return rgb_mycolor

def cellMaskColor(adata, maskFile, outDir, anno='cell2loc_anno'):
    
    maskImg = tifffile.imread(maskFile)
    if (maskImg.max() == 1):
        _, labels = cv2.connectedComponents(maskImg)
    else:
        labels = maskImg
    dst = np.nonzero(labels)
    paletteKey = f"{anno}_colors"
    mycolor = list(adata.uns[paletteKey]) if paletteKey in adata.uns.keys() else None
    rgbColor = _getColorList(mycolor)
    clusterDict = dict(zip(adata.obs.index.astype(int), adata.obs[anno].cat.codes))
    imgColors = [rgbColor[int(clusterDict[x])] if x in clusterDict.keys() else [255, 255, 255] for x in labels[dst]]
    newImg = np.zeros((maskImg.shape[0], maskImg.shape[1], 3))
    newImg[dst] = imgColors
    newImg = newImg.astype(int)

    if not os.path.exists(outDir):
        os.makedirs(outDir, exist_ok=True)
    cellMaskColoredFile = os.path.join(outDir, "cellMask_{0}.png".format(anno))
    cv2.imwrite(cellMaskColoredFile, newImg)

    #get legend
    cells = adata.obs[anno].cat.categories
    width = 10000
    high = 1000
    legend = np.zeros([high*len(cells), width,3])
    legend.fill(255)
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 20  
    # Line thickness of 40 px
    thickness = 50
    pointSize = 100
    for i in range(len(cells)):
        color = rgbColor[i]
        cell = cells[i]
        point = (int(width/8), int((i+0.4)*high))
        cv2.circle(legend, point, pointSize, color, pointSize*2)
        org = (int(width/5), int((i+0.6)*high))
        legend = cv2.putText(legend, cell, org, font, fontScale, (0, 0, 0), thickness, cv2.LINE_AA)
    legendFile = os.path.join(outDir, "cellMask_{0}_legend.jpg".format(anno))
    cv2.imwrite(legendFile, legend)
    return 0

def cell_segmentation(geneExpFile, cellMask, tissueMask, outFile):
    
    columntypes = {"geneID": object, "x": np.uint32, "y": np.uint32, "UMICount": np.uint16, "MIDCount": np.uint16, "label": np.uint32, "tag": "category"}
    df = pd.read_csv(geneExpFile, sep="\t", dtype=columntypes, quoting=csv.QUOTE_NONE, comment="#")
    if "MIDCounts" in df.columns:
        df.rename(columns={"MIDCounts": "UMICount"}, inplace=True)
    elif 'values' in df.columns:
        df.rename(columns={"values": "UMICount"}, inplace=True)
    elif 'MIDCount' in df.columns:
        df.rename(columns={'MIDCount': 'UMICount'}, inplace=True)
    offsetX, offsetY = getOffset(geneExpFile)
    df['x'] = df['x'] + offsetX
    df['y'] = df['y'] + offsetY

    if tissueMask != None:
        tissue_mask = tifffile.imread(tissueMask)
        maxY, maxX = tissue_mask.shape
        if maxX > df['x'].max() or maxY > df['y'].max():
            print ("WARMING: mask is out of bounds")
            df = df.loc[(df['x'] < maxX)&(df['y'] < maxY)]
        df = df.loc[tissue_mask[df['y'], df['x']] > 0]
    
    mask = cv2.imread(cellMask, -1)
    _, labels = cv2.connectedComponents(mask)
    tissuedf = pd.DataFrame()
    dst = np.nonzero(labels)

    tissuedf['x'] = dst[1]
    tissuedf['y'] = dst[0]
    tissuedf['label'] = labels[dst]

    res = pd.merge(df, tissuedf, on=['x', 'y'], how='inner')
    res.to_csv(outFile, sep="\t", index=None)

In [ ]:
outDir = "./out"
bin_type = 'cell'

cellbin_gem_file = os.path.join(outDir, f"cellbin.gem.gz")

if not os.path.exists(cellbin_gem_file):
    utils.cell_segmentation(gemFile, maskFile, tissueMask, cellbin_gem_file)
    gemFile = cellbin_gem_file
    
adata = utils.cellbin2h5ad(gemFile)

# execute cell cluster by scanpy leiden
clusterAdataFile = os.path.join(outDir, f"cellCluster_{bin_type}.h5ad")
cellCluster = CellCluster(clusterAdataFile)
cellCluster.scanpy_cluster(adata)

#execute cell annotation by cell2location based on cell report reference
gpu = 0
N_cells_per_location = 1
refFile = os.path.join(outDir, "inf_aver.csv")
cell2locationAnno = Cell2locationAnno(refFile, clusterAdataFile, outDir, gpu, bin_type=bin_type)
adata = cell2locationAnno.run_anno(N_cells_per_location)